In [9]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import rand
import time
import torch
import os
#=========================================================================================
torch.set_default_tensor_type(torch.DoubleTensor)

class fit():
    def __init__(self,epsilon = 0.9, learning_rate = 0.1, max_iter = 151,regularizer = 0.):
        self.epsilon = epsilon
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.regularizer = regularizer
        
        
    def operators(self,configuration):
        #O = <S_i S_j>
        system_size,sample_size = configuration.shape
        Op = np.zeros((int(system_size*(system_size-1)/2), sample_size))

        jindex = 0
        for index in range(system_size-1):
            temp = configuration[index,:] * configuration[index+1:,:]
            Op[jindex: jindex + system_size - index -1 , :] = temp
            jindex += system_size - index - 1
        return Op
        
        
    def em_cpu(self, ops):
        n_ops,n_samples = ops.shape
        np.random.seed(13)
        w = np.random.rand(n_ops)-0.5
        for i in range(self.max_iter):

            energies_w = w@ops
            
            energies_max = energies_w.max()  
            probs_w = np.exp((energies_w-energies_max)*(self.epsilon-1)) 
            probs_w /= np.sum(probs_w/1000) ### avoid too large value //empiricalli choice 1000
            probs_w /= 1000
            ops_expect_w = np.sum(probs_w[np.newaxis,:]*ops,axis=1)


            w += self.learning_rate*(ops_expect_w - w*self.epsilon - self.regularizer*w)

        return w
    
    def em_gpu(self, ops):
        
        n_ops = ops.shape[0]
        np.random.seed(13)
        w = np.random.rand(n_ops)-0.5
        w = torch.from_numpy(w).cuda()

        ops = torch.from_numpy(ops).cuda()
        
        for i in range(self.max_iter):

            energies_w = w@ops

            probs_w = torch.exp((energies_w)*(self.epsilon-1)) 
            z_data = torch.sum(probs_w)
            probs_w /= z_data
            ops_expect_w = torch.sum(probs_w[np.newaxis,:]*ops,axis=1)


            w += self.learning_rate*(ops_expect_w - w*self.epsilon - self.regularizer*w)

        return w


In [10]:
T = np.linspace(5.,0.001,50)
L_arr = [320]
sample_size_arr = [10000]
ens = 10
model = 'SK model'
em = fit()


MSE_arr = np.zeros((len(T),ens))


for L in L_arr:

    
    for sample_size in sample_size_arr:
        
#---------------------------------------------------------------------------------------------------------------------

        YOUR_DIRECTORY_NAME = r'E:\Study\erasure_machine\save_result\L{0}_{1}\\'.format(L,sample_size)
        
        try:
            if not(os.path.exists(YOUR_DIRECTORY_NAME)):
                os.makedirs(YOUR_DIRECTORY_NAME)
        except OSError as e:
            if OSError:
                print("Failed to create directory")
#---------------------------------------------------------------------------------------------------------------------
        for en in range(ens):

            config_arr = np.zeros((L,1,len(T)))
            
            start = time.time()
            path = r'E:\Study\erasure_machine\L{0}\L{0}_{1}\\'.format(L,en)
            for r in range(50):
                config_arr_0 = np.load(path + 'config_%d.npy'%(r))
                config_arr = np.concatenate((config_arr,config_arr_0),axis = 1)
                
            randidx = np.random.choice(50000,sample_size,replace = False)
            config_arr = config_arr[:,1:,:]
            config_arr = config_arr[:,randidx,:]
            

#---------------------------------------------------------------------------------------------------------------------
            w_mat = np.load(path + 'J.npy')
            iu1 = np.triu_indices(L,1)
            w_true = w_mat[iu1]
            #w_true = torch.from_numpy(w_true).to('cuda')
            
#---------------------------------------------------------------------------------------------------------------------
            for i,t in enumerate(T):

                ops = em.operators(config_arr[:,:,i])
                w = em.em_cpu(ops)
                    
                MSE_arr[i,en] = np.sum((w_true-w)**2) / np.sum((w_true)**2)


                YOUR_DIRECTORY_NAME_en =  r'E:\Study\erasure_machine\save_result\L{0}_{1}\en{2}\t{3}\\'.format(L,sample_size,en,t)
                try:
                    if not(os.path.exists(YOUR_DIRECTORY_NAME_en)):
                        os.makedirs(YOUR_DIRECTORY_NAME_en)
                except OSError as e:
                    if OSError:
                        print("Failed to create directory")

                np.save(YOUR_DIRECTORY_NAME_en + 'w_true.npy',w_true)
                np.save(YOUR_DIRECTORY_NAME_en + 'w_infered.npy',w)
                
                
                print(t , " = " , MSE_arr[i,en])
                
            np.save(YOUR_DIRECTORY_NAME + 'MSE_EM.npy',MSE_arr)
            print(en,'/',ens)
            print(" time :", time.time() - start)

#---------------------------------------------------------------------------------------------------------------------        
        
        std_arr_em = np.zeros(len(T))
        mean_arr_em = np.zeros(len(T))

        
        for t in range(len(T)):
            std_arr_em[t] = MSE_arr[t,:].std()
            mean_arr_em[t] = np.average(MSE_arr[t,:])


        plt.figure(figsize=(15,10))
        plt.errorbar(T,mean_arr_em,2*std_arr_em/np.sqrt(ens-1),capsize=3,ecolor='k',fmt="ro-",elinewidth=0.5,label='N : {0}'.format(L))
        
        np.save( YOUR_DIRECTORY_NAME + 'MSE_EM.npy',mean_arr_em)
        np.save( YOUR_DIRECTORY_NAME + 'Errorbar.npy',2*std_arr_em/np.sqrt(ens-1))
        np.save( YOUR_DIRECTORY_NAME + 'T.npy',T)

        plt.yscale('log')
        plt.ylabel('MSE')
        plt.xlabel('Temperature')


        plt.legend()
        plt.grid(1)
        plt.savefig( YOUR_DIRECTORY_NAME + 'figure.svg')
        plt.savefig( YOUR_DIRECTORY_NAME + 'figure.png')
        plt.show()
        

5.0  =  0.6682727329986881
4.897979591836735  =  0.6642475313304638
4.795959183673469  =  0.6562408316833691
4.693938775510204  =  0.649911600548113
4.591918367346938  =  0.6415622267069976
4.489897959183674  =  0.6339635290269162
4.387877551020408  =  0.6272392105047706
4.285857142857143  =  0.618128154612873
4.183836734693878  =  0.6095237911022211
4.081816326530612  =  0.600365405850609
3.979795918367347  =  0.5882350020015825
3.877775510204082  =  0.5817358038069598
3.7757551020408164  =  0.5723893581015705
3.673734693877551  =  0.5618634243836623
3.571714285714286  =  0.5502452917830444
3.4696938775510207  =  0.5402060525557084
3.3676734693877552  =  0.5257233261860562
3.26565306122449  =  0.5179559421592532
3.1636326530612244  =  0.504326256711356
3.0616122448979595  =  0.4926742516895335
2.959591836734694  =  0.47831848129701454
2.8575714285714287  =  0.46669483636799075
2.7555510204081637  =  0.4527789056374842
2.6535306122448983  =  0.43629038862957537
2.551510204081633  =  0.

KeyboardInterrupt: 